In [ ]:
import requests
import logging
import nest_asyncio
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes


nest_asyncio.apply()

# Настройки
LM_STUDIO_API_URL = "http://127.0.0.1:1234/v1/chat/completions"
TELEGRAM_BOT_TOKEN = ""


user_contexts = {}

# Настройка логирования
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик команды /start"""
    user_id = update.message.from_user.id
    user_contexts[user_id] = []
    
    welcome_text = (
        "Привет! Я бот с поддержкой контекста на основе локальной LLM.\n"
        "Я помню нашу беседу и могу отвечать с учетом предыдущих сообщений.\n"
        "Используйте /clear чтобы очистить историю диалога."
    )
    await update.message.reply_text(welcome_text)

async def clear_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик команды /clear - очищает контекст пользователя"""
    user_id = update.message.from_user.id
    user_contexts[user_id] = []
    await update.message.reply_text("История диалога очищена. Начинаем новый разговор!")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик текстовых сообщений"""
    user_id = update.message.from_user.id
    user_message = update.message.text
    
    if user_id not in user_contexts:
        user_contexts[user_id] = []
    
    user_contexts[user_id].append({"role": "user", "content": user_message})
    
    try:
        response = generate_response(user_contexts[user_id])
        user_contexts[user_id].append({"role": "assistant", "content": response})
        
        await update.message.reply_text(response)
        
    except Exception as e:
        error_message = f"Произошла ошибка при обработке запроса: {str(e)}"
        logging.error(error_message)
        await update.message.reply_text("Извините, произошла ошибка. Попробуйте позже.")

def generate_response(conversation_history):
    """Генерация ответа через LM Studio API"""
    payload = {
        "messages": conversation_history,
        "temperature": 0.7,
        "max_tokens": 512,
        "stream": False
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(LM_STUDIO_API_URL, json=payload, headers=headers, timeout=30)
    
    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        raise Exception(f"API error: {response.status_code} - {response.text}")

def main():
    """Основная функция запуска бота"""
    application = Application.builder().token(TELEGRAM_BOT_TOKEN).build()
    
    application.add_handler(CommandHandler("start", start_command))
    application.add_handler(CommandHandler("clear", clear_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    
    print("Бот запущен...")
    application.run_polling()

if __name__ == "__main__":
    main()

Бот запущен...


2025-11-30 18:22:20,155 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8550935246:AAGtK5u0XHNGHhLizQiiknnyl2geH6EWI10/getMe "HTTP/1.1 200 OK"
2025-11-30 18:22:20,227 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8550935246:AAGtK5u0XHNGHhLizQiiknnyl2geH6EWI10/deleteWebhook "HTTP/1.1 200 OK"
2025-11-30 18:22:20,229 - telegram.ext.Application - INFO - Application started
2025-11-30 18:22:24,033 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8550935246:AAGtK5u0XHNGHhLizQiiknnyl2geH6EWI10/getUpdates "HTTP/1.1 200 OK"
2025-11-30 18:22:24,889 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8550935246:AAGtK5u0XHNGHhLizQiiknnyl2geH6EWI10/sendMessage "HTTP/1.1 200 OK"
2025-11-30 18:22:29,298 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8550935246:AAGtK5u0XHNGHhLizQiiknnyl2geH6EWI10/getUpdates "HTTP/1.1 200 OK"
2025-11-30 18:22:30,256 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8550935246:AA